In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import time
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
torch.cuda.empty_cache()

# Param

In [4]:
BATCH_SIZE = 100
EPOCHS = 5
LEARNING_RATE = 1e-4
NUM_CLASSES = 15
IMG_SIZE = 224

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_dataset = datasets.ImageFolder(
    root='/content/drive/MyDrive/crop_disease_prediction/data/train',
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root='/content/drive/MyDrive/crop_disease_prediction/data/valid',
    transform=val_test_transform
)

test_dataset = datasets.ImageFolder(
    root='/content/drive/MyDrive/crop_disease_prediction/data/test',
    transform=val_test_transform
)




In [8]:
len(train_dataset),len(val_dataset),len(test_dataset)

(13104, 300, 750)

In [9]:
train_dataset.classes

['Aphid',
 'Black Rust',
 'Blast',
 'Brown Rust',
 'Common Root Rot',
 'Fusarium Head Blight',
 'Healthy',
 'Leaf Blight',
 'Mildew',
 'Mite',
 'Septoria',
 'Smut',
 'Stem fly',
 'Tan spot',
 'Yellow Rust']

In [10]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# Load Model

In [11]:
class EfficientNetV2(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(EfficientNetV2, self).__init__()
        self.model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)

        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [12]:
class ConvNeXt(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(ConvNeXt, self).__init__()
        self.model = models.convnext_large(weights=models.ConvNeXt_Large_Weights.IMAGENET1K_V1)
        # Freeze all parameters first
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze the final two classifier blocks
        for param in self.model.classifier[-3:].parameters():
            param.requires_grad = True

        num_ftrs = self.model.classifier[-1].in_features
        self.model.classifier = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LayerNorm((num_ftrs,), eps=1e-06, elementwise_affine=True),
            nn.Linear(num_ftrs, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [13]:
class ResNet50V2(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(ResNet50V2, self).__init__()
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # Freeze all layers first
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze layer4 (last block) and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Function Train Eval Plot

In [14]:
def train_model(model, model_name, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS):
    print(f"\n--- Training {model_name} ---")
    start_time = time.time()

    best_val_accuracy = 0.0
    best_model_path = f"best_{model_name}.pth"

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        batch = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            print("epoch:",epoch+1, " Batch:",batch+1)
            batch = batch+1

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = running_corrects.double() / len(train_loader.dataset)
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc.item())

        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = running_corrects.double() / len(val_loader.dataset)
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc.item())

        epoch_time = time.time() - epoch_start_time
        print(f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f} | "
              f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f} | Time: {epoch_time:.2f}s")

        # Save best model
        if epoch_val_acc > best_val_accuracy:
            best_val_accuracy = epoch_val_acc
            torch.save(model.state_dict(), best_model_path)

    total_time = time.time() - start_time
    print(f"Training {model_name} complete in {total_time // 60:.0f}m {total_time % 60:.0f}s")
    print(f"Best Val Acc for {model_name}: {best_val_accuracy:.4f}")

    return history

In [15]:
def evaluate_model(model, model_name, test_loader, class_names):
    print(f"\n--- Evaluating {model_name} ---")
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1_score, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0
    )

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision (weighted): {precision:.4f}")
    print(f"Test Recall (weighted): {recall:.4f}")
    print(f"Test F1-score (weighted): {f1_score:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.savefig(f"confusion_matrix_{model_name}.png")
    plt.close()

    return {
        "model_name": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

In [16]:
def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss vs. Epochs - {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title(f'Accuracy vs. Epochs - {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f"training_history_{model_name}.png")
    plt.close()

# Train Model

In [17]:
architectures_to_train = {
    "ConvNeXt": ConvNeXt,
    "ResNet50V2": ResNet50V2
}

In [18]:
all_results = []
class_names = train_dataset.classes
for model_name, model_class in architectures_to_train.items():
    model = model_class(num_classes=NUM_CLASSES).to(device)
    best_model_path = f"best_{model_name}.pth"

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

    history = train_model(model, model_name, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS)
    plot_training_history(history, model_name)

    print(f"Loading best model for evaluation.")
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    eval_metrics = evaluate_model(model, model_name, test_loader, class_names)
    eval_metrics['best_model_path'] = best_model_path
    all_results.append(eval_metrics)

results_df = pd.DataFrame(all_results)
results_df.to_csv("models_evaluation.csv", index=False)
print("\n--- All models trained and evaluated. ---")
print(results_df)

Downloading: "https://download.pytorch.org/models/convnext_large-ea097f82.pth" to /root/.cache/torch/hub/checkpoints/convnext_large-ea097f82.pth


100%|██████████| 755M/755M [00:07<00:00, 106MB/s]



--- Training ConvNeXt ---
Epoch 1/5
epoch: 1  Batch: 1
epoch: 1  Batch: 2
epoch: 1  Batch: 3
epoch: 1  Batch: 4
epoch: 1  Batch: 5
epoch: 1  Batch: 6
epoch: 1  Batch: 7
epoch: 1  Batch: 8
epoch: 1  Batch: 9
epoch: 1  Batch: 10
epoch: 1  Batch: 11
epoch: 1  Batch: 12
epoch: 1  Batch: 13
epoch: 1  Batch: 14
epoch: 1  Batch: 15
epoch: 1  Batch: 16
epoch: 1  Batch: 17
epoch: 1  Batch: 18
epoch: 1  Batch: 19
epoch: 1  Batch: 20
epoch: 1  Batch: 21
epoch: 1  Batch: 22
epoch: 1  Batch: 23
epoch: 1  Batch: 24
epoch: 1  Batch: 25
epoch: 1  Batch: 26
epoch: 1  Batch: 27
epoch: 1  Batch: 28
epoch: 1  Batch: 29
epoch: 1  Batch: 30
epoch: 1  Batch: 31
epoch: 1  Batch: 32
epoch: 1  Batch: 33
epoch: 1  Batch: 34
epoch: 1  Batch: 35
epoch: 1  Batch: 36
epoch: 1  Batch: 37
epoch: 1  Batch: 38
epoch: 1  Batch: 39
epoch: 1  Batch: 40
epoch: 1  Batch: 41
epoch: 1  Batch: 42
epoch: 1  Batch: 43
epoch: 1  Batch: 44
epoch: 1  Batch: 45
epoch: 1  Batch: 46
epoch: 1  Batch: 47
epoch: 1  Batch: 48
epoch: 1  Ba

100%|██████████| 97.8M/97.8M [00:00<00:00, 182MB/s]



--- Training ResNet50V2 ---
Epoch 1/5
epoch: 1  Batch: 1
epoch: 1  Batch: 2
epoch: 1  Batch: 3
epoch: 1  Batch: 4
epoch: 1  Batch: 5
epoch: 1  Batch: 6
epoch: 1  Batch: 7
epoch: 1  Batch: 8
epoch: 1  Batch: 9
epoch: 1  Batch: 10
epoch: 1  Batch: 11
epoch: 1  Batch: 12
epoch: 1  Batch: 13
epoch: 1  Batch: 14
epoch: 1  Batch: 15
epoch: 1  Batch: 16
epoch: 1  Batch: 17
epoch: 1  Batch: 18
epoch: 1  Batch: 19
epoch: 1  Batch: 20
epoch: 1  Batch: 21
epoch: 1  Batch: 22
epoch: 1  Batch: 23
epoch: 1  Batch: 24
epoch: 1  Batch: 25
epoch: 1  Batch: 26
epoch: 1  Batch: 27
epoch: 1  Batch: 28
epoch: 1  Batch: 29
epoch: 1  Batch: 30
epoch: 1  Batch: 31
epoch: 1  Batch: 32
epoch: 1  Batch: 33
epoch: 1  Batch: 34
epoch: 1  Batch: 35
epoch: 1  Batch: 36
epoch: 1  Batch: 37
epoch: 1  Batch: 38
epoch: 1  Batch: 39
epoch: 1  Batch: 40
epoch: 1  Batch: 41
epoch: 1  Batch: 42
epoch: 1  Batch: 43
epoch: 1  Batch: 44
epoch: 1  Batch: 45
epoch: 1  Batch: 46
epoch: 1  Batch: 47
epoch: 1  Batch: 48
epoch: 1  